#### Spam Ham Project

In [2]:
import pandas as pd
meessages = pd.read_csv('spam1.csv',encoding='latin-1')

In [4]:
meessages = meessages.iloc[:,0:2]

In [6]:
meessages.rename(columns={'v1':'label','v2':'messages'},inplace=True)

C:\Users\DIVYANSHU\AppData\Local\Temp\ipykernel_1640\1481446770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meessages.rename(columns={'v1':'label','v2':'messages'},inplace=True)


In [7]:
meessages

,label,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
## Data cleaning and preprocessing 
import re
from nltk.corpus import stopwords

In [9]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [13]:
meessages.shape

(5572, 2)

In [12]:
corpus = []
for i in range(0,len(meessages)):
    review = re.sub('[^a-zA-Z]',' ',meessages['messages'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [14]:
## creating bag of words moodel
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,2),max_features=2500)

In [16]:
X = cv.fit_transform(corpus).toarray()

In [17]:
import numpy as np
np.set_printoptions(edgeitems=30,linewidth=100000,formatter=dict(float=lambda x:"%.3g" % X))

In [19]:
cv.vocabulary_

{'go': 801,
 'point': 1605,
 'crazi': 452,
 'avail': 123,
 'bugi': 230,
 'great': 855,
 'world': 2430,
 'la': 1084,
 'cine': 349,
 'got': 843,
 'wat': 2338,
 'ok': 1472,
 'lar': 1096,
 'joke': 1051,
 'wif': 2395,
 'free': 723,
 'entri': 617,
 'wkli': 2419,
 'comp': 400,
 'win': 2400,
 'fa': 653,
 'cup': 466,
 'final': 688,
 'st': 1961,
 'may': 1265,
 'text': 2074,
 'receiv': 1699,
 'question': 1664,
 'std': 1976,
 'txt': 2203,
 'rate': 1679,
 'appli': 86,
 'free entri': 728,
 'entri wkli': 619,
 'std txt': 1977,
 'txt rate': 2209,
 'rate appli': 1680,
 'dun': 577,
 'say': 1795,
 'earli': 583,
 'alreadi': 55,
 'nah': 1386,
 'think': 2100,
 'goe': 823,
 'usf': 2273,
 'live': 1163,
 'around': 99,
 'though': 2110,
 'freemsg': 735,
 'hey': 921,
 'darl': 483,
 'week': 2363,
 'word': 2425,
 'back': 140,
 'like': 1148,
 'fun': 755,
 'still': 1979,
 'tb': 2052,
 'xxx': 2465,
 'send': 1826,
 'even': 627,
 'brother': 220,
 'speak': 1942,
 'treat': 2178,
 'per': 1543,
 'request': 1733,
 'mell': 12

In [ ]:
## independent features (output)
y = pd.get_dummies(meessages['label'])

In [24]:
y

,ham,spam
0,True,False
1,True,False
2,False,True
3,True,False
4,True,False
...,...,...
5567,False,True
5568,True,False
5569,True,False
5570,True,False


In [25]:
y = y.iloc[:,0]

In [26]:
y

0        True
1        True
2       False
3        True
4        True
        ...  
5567    False
5568     True
5569     True
5570     True
5571     True
Name: ham, Length: 5572, dtype: bool

In [27]:
## train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20)

In [28]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB()

In [29]:
spam_detect_model.fit(X_train,y_train)

MultinomialNB()

In [30]:
y_pred = spam_detect_model.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9811659192825112
              precision    recall  f1-score   support

       False       0.95      0.91      0.93       149
        True       0.99      0.99      0.99       966

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [32]:
## creating TF-IDF model and prediction using this
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf =TfidfVectorizer(max_features=2500,ngram_range=(1,2))

In [33]:
X = tfidf.fit_transform(corpus).toarray()

In [34]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20)

In [35]:
spam_detect_model.fit(X_train,y_train)
y_pred = spam_detect_model.predict(X_test)

In [36]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))


0.9695067264573991
              precision    recall  f1-score   support

       False       1.00      0.77      0.87       151
        True       0.97      1.00      0.98       964

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115

